# Two-Way_ANOVA_with_Python

**Background: **In this notebook, I will be calculating the F-score for the two-way ANOVA for the fecal fat data. These data are the fat in grams per day from one of six subjects, measured after they received one of four different pill types. I have already performed this analysis using a one-way ANOVA F-test using an R kernel, which I will repeat here, but defining the details of a two-way ANOVA test in R is quite complex for such a simple test.

**Purpose: **To use statsmodels to perform this test. 

**Methods: **First, I will load the data. Then, I will format it for use with stats models. Then, I will run a one-way ANOVA test using stats models and verify that I get the same answer as I did using R. Then, I will run a two-way ANOVA test on the data using statsmodels and confirm my answer with the textbook. 

**Conclusions: **
* These results match the book results, so I know that the one-way ANOVA is working properly.
* These results match those given in the book, except that these results don't include the F-score/p-value for the model itself.

# Inits

## Imports

In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

## Definitions

## Funcs

# Generate Data

## Load data

In [2]:
fecfat_df = pd.read_csv('./data/fecfat.csv')

## Format for use with statsmodels

In [3]:
fecfat_df['subject'] = fecfat_df['subject'].astype(str)

In [4]:
fecfat_df.head()

,fecfat,subject,pilltype
0,44.5,1,none
1,7.3,1,tablet
2,3.4,1,capsule
3,12.4,1,coated
4,33.0,2,none


# One-Way ANOVA

## Run a one-way ANOVA test using stats models

In [5]:
one_way_anova_formula = 'fecfat ~ C(pilltype)'

In [6]:
one_way_model = ols(formula=one_way_anova_formula, data = fecfat_df).fit()
one_way_aov_table = anova_lm(one_way_model, typ=1)

In [7]:
one_way_aov_table

,df,sum_sq,mean_sq,F,PR(>F)
C(pilltype),3.0,2008.601702,669.533901,1.861532,0.168656
Residual,20.0,7193.363277,359.668164,NaN,NaN


In [23]:
one_way_model.f_test()

<bound method LikelihoodModelResults.f_test of <statsmodels.regression.linear_model.OLSResults object at 0x109821438>>

In [69]:
one_way_model.normalized_cov_params

,Intercept,C(pilltype)[T.coated],C(pilltype)[T.none],C(pilltype)[T.tablet]
Intercept,0.166667,-0.166667,-0.166667,-0.166667
C(pilltype)[T.coated],-0.166667,0.333333,0.166667,0.166667
C(pilltype)[T.none],-0.166667,0.166667,0.333333,0.166667
C(pilltype)[T.tablet],-0.166667,0.166667,0.166667,0.333333


In [63]:
one_way_model.cov_params() / one_way_model.normalized_cov_params

,Intercept,C(pilltype)[T.coated],C(pilltype)[T.none],C(pilltype)[T.tablet]
Intercept,359.668164,359.668164,359.668164,359.668164
C(pilltype)[T.coated],359.668164,359.668164,359.668164,359.668164
C(pilltype)[T.none],359.668164,359.668164,359.668164,359.668164
C(pilltype)[T.tablet],359.668164,359.668164,359.668164,359.668164


In [42]:
robust_cov = one_way_model.cov_params()
robust_cov

,Intercept,C(pilltype)[T.coated],C(pilltype)[T.none],C(pilltype)[T.tablet]
Intercept,59.944694,-59.944694,-59.944694,-59.944694
C(pilltype)[T.coated],-59.944694,119.889388,59.944694,59.944694
C(pilltype)[T.none],-59.944694,59.944694,119.889388,59.944694
C(pilltype)[T.tablet],-59.944694,59.944694,59.944694,119.889388


In [32]:
fecfat_pivot_df = fecfat_df.pivot_table(values='fecfat', index='subject', columns='pilltype')

In [67]:
(fecfat_df['fecfat'] - fecfat_df['fecfat'].mean()).map(lambda x: x ** 2).sum()

9201.964979157472

In [56]:
(fecfat_pivot_df['capsule'] - fecfat_pivot_df['capsule'].mean()).map(lambda x: x ** 2).sum()

836.8883415412893

In [29]:
robust_cov

,Intercept,C(pilltype)[T.coated],C(pilltype)[T.none],C(pilltype)[T.tablet]
Intercept,59.944694,-59.944694,-59.944694,-59.944694
C(pilltype)[T.coated],-59.944694,119.889388,59.944694,59.944694
C(pilltype)[T.none],-59.944694,59.944694,119.889388,59.944694
C(pilltype)[T.tablet],-59.944694,59.944694,59.944694,119.889388


In [27]:
from statsmodels.formula.formulatools import (_remove_intercept_patsy,
                                    _has_intercept, _intercept_idx)
from statsmodels.compat.python import lrange, lmap
design_info = one_way_model.model.data.design_info
terms_info = design_info.terms[:]
terms_info = _remove_intercept_patsy(terms_info)

robust_cov = one_way_model.cov_params()

for i, term in enumerate(terms_info):
    cols = design_info.slice(term)
    # grab all varaibles except interaction effects that contain term
    # need two hypotheses matrices L1 is most restrictive, ie., term==0
    # L2 is everything except term==0
    cols = design_info.slice(term)
    L1 = lrange(cols.start, cols.stop)
    L2 = []
    term_set = set(term.factors)
    for t in terms_info: # for the term you have
        other_set = set(t.factors)
        if term_set.issubset(other_set) and not term_set == other_set:
            col = design_info.slice(t)
            # on a higher order term containing current `term`
            L1.extend(lrange(col.start, col.stop))
            L2.extend(lrange(col.start, col.stop))

    L1 = np.eye(one_way_model.model.exog.shape[1])[L1]
    L2 = np.eye(one_way_model.model.exog.shape[1])[L2]

    if L2.size:
        LVL = np.dot(np.dot(L1,robust_cov),L2.T)
        from scipy import linalg
        orth_compl,_ = linalg.qr(LVL)
        r = L1.shape[0] - L2.shape[0]
        # L1|2
        # use the non-unique orthogonal completion since L12 is rank r
        L12 = np.dot(orth_compl[:,-r:].T, L1)
    else:
        L12 = L1
        r = L1.shape[0]
    #from IPython.core.debugger import Pdb; Pdb().set_trace()

    f = one_way_model.f_test(L12, cov_p=robust_cov)
    table.loc[table.index[i], test] = test_value = f.fvalue
    table.loc[table.index[i], pr_test] = f.pvalue    

NameError: name 'robust_cov' is not defined

## Verify answer

**These results match the book results, so I know that the one-way ANOVA is working properly.**

# Two-Way ANOVA

## Run a two-way ANOVA test using statsmodels 

In [17]:
two_way_anova_formula = 'fecfat ~ C(pilltype) + C(subject)'
two_way_model = ols(formula=two_way_anova_formula, data = fecfat_df).fit()
two_way_aov_table = anova_lm(two_way_model, typ=2)
two_way_aov_table

,sum_sq,df,F,PR(>F)
C(pilltype),2008.601702,3.0,6.257391,0.005741
C(subject),5588.379959,5.0,10.445679,0.000182
Residual,1604.983318,15.0,NaN,NaN


## Verify answer

**These results match those given in the book, except that these results don't include the F-score/p-value for the model itself.**